In [1]:
# conda create -n sss python=3.8 -y && conda activate sss
# conda install -y -c bioconda gffutils jupyter tqdm cyvcf2 pathlib2 pandarallel pysam liftover pybedtools

import os
import re
import numpy as np
import pandas as pd
# from Bio.Seq import Seq
# from liftover import get_lifter
from pathlib2 import Path
from pandarallel import pandarallel
from tqdm import tqdm
import gffutils
import pysam
from cyvcf2 import VCF

### Logging setup
from logging import getLogger, config
import yaml
parent_directory = os.path.dirname(os.path.dirname('__file__'))
config_path: str = os.path.join(parent_directory, '../../../config/logging.yaml')
with open(config_path, 'r') as f:
    config.dictConfig(yaml.safe_load(f))
logger = getLogger(__name__)

########   Initialize and setup pandas methods   ########
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp' 
pandarallel.initialize(nb_workers=6, progress_bar=True, verbose=1, use_memory_fs=False) 
tqdm.pandas()

import sys
try: 
    __file__
    sys.path.append(os.path.join(os.path.dirname('__file__')))
except NameError:
    Path().resolve()
    sys.path.append(os.path.join(Path().resolve(), '../../../'))

from libs import utils, preprocess, variantfilter, posparser, splaiparser
# from libs import predeffect, scoring
from libs import anno_spliceai, anno_clinvar
from libs.deco import print_filtering_count
# from libs import predeffect
from libs.scoring import Scoring
from libs import predeffect


gencode_gff = '../../../Resources/05_GENCODE_v43lift37/gencode.v43lift37.annotation.sort.gff3.gz'

try:
    db_anno_gencode = '../../../Resources/06_gffutilsdb/gencode.v43lift37.annotation.gtf.db'
    db_anno_intron = '../../../Resources/06_gffutilsdb/gencode.v43lift37.annotation.intron.gtf.db'
    db = gffutils.FeatureDB(db_anno_gencode)
    db_intron = gffutils.FeatureDB(db_anno_intron)
except ValueError:
    db_anno_gencode = '/resources/DBs/gencode.v43lift37.annotation.gtf.db'
    db_anno_intron = '/resources/DBs/gencode.v43lift37.annotation.intron.gtf.db'
    db = gffutils.FeatureDB(db_anno_gencode)
    db_intron = gffutils.FeatureDB(db_anno_intron)

## Thresholds configuration
thresholds_SpliceAI_parser: dict = {
    'TH_min_sALDL': 0.02, 'TH_max_sALDL': 0.2, 
    'TH_min_sAGDG': 0.01, 'TH_max_sAGDG': 0.05,
    'TH_min_GExon': 25, 'TH_max_GExon': 500,
    'TH_sAG': 0.2, 'TH_sDG': 0.2
    }

#### 前処理．SQLから抽出したallmut.csvを編集する．

In [ ]:
all_mut_default_colnames: list = [
    "disase", "gene", "chrom", "genename", "gdbid", "omimid", "amino", 
    "deletion", "insertion", "codon", "codonAff", "descr", "refseq", "hgvs", 
    "hgvsAll", "dbsnp", "chromosome", "startCoord", "endCoord", 
    "expected_inheritance", "gnomad_AC", "gnomad_AF", "gnomad_AN", "tag", 
    "dmsupport", "rankscore", "mutype", "author", "title", "fullname", 
    "allname", "vol", "page", "year", "pmid", "pmidAll", "reftag", "comments", 
    "acc_num", "new_date", "base", "clinvarID", "clinvar_clnsig"
]

allmut: pd.DataFrame = pd.read_csv(
    'allmut.csv', sep=';', encoding='cp1252', names=all_mut_default_colnames, 
    skiprows=1,low_memory=False)


allmut: pd.DataFrame = pd.read_csv(
    'allmut.csv', sep=';', encoding='cp1252', names=all_mut_default_colnames, 
    skiprows=1,low_memory=False)

allmut = allmut[
    ["gene", "genename", "mutype", "clinvar_clnsig", "tag",
     "refseq", "hgvs", "hgvsAll", "chromosome", "startCoord", "endCoord", 
     "amino", "deletion", "insertion", "expected_inheritance", "gnomad_AF"]]

# Drop non-numeric values in 'startCoord'
allmut = allmut.dropna(subset=['startCoord'])

# Drop duplicates in 'chrom', 'startCoord', and 'endCoord'
allmut = allmut.drop_duplicates(subset=['chromosome', 'startCoord', 'endCoord'])

# Extract tag == "DM" from allmut
allmut_dm = allmut[allmut.tag == "DM"].copy()
print(f"A total of {len(allmut_dm)} DM mutations are found in allmut.")

allmut_dm['startCoord'] = allmut_dm['startCoord'].astype(int)
allmut_dm = allmut_dm.rename(columns={'chromosome': 'CHROM', 'startCoord': 'POS_hg38'})

# Fillna with empty string in "gnomad_AF" colmun in allmut_dm
# Extratct MAF 0 from allmut_dm
allmut_dm['gnomad_AF'].fillna(0, inplace=True)
allmut_dm_maf0 = allmut_dm[allmut_dm['gnomad_AF'] == 0].copy()
print(f"A total of {len(allmut_dm_maf0)} DM mutations are found in allmut with MAF 0.")

# Extract non-deletion or non-insertion from allmut_dm
allmut_dm_maf0_snv = allmut_dm_maf0[(allmut_dm_maf0['deletion'].isnull()) & (allmut_dm_maf0['insertion'].isnull())]
print(f"A total of {len(allmut_dm_maf0_snv)} DM mutations are found in allmut with MAF 0 and non-deletion or non-insertion.")

# Extract the mutation type from the mutype column
splice_mutations = allmut_dm_maf0_snv[allmut_dm_maf0_snv["mutype"].str.contains("splice")].copy()
non_splice_mutations = allmut_dm_maf0_snv[~allmut_dm_maf0_snv["mutype"].str.contains("splice")]
print(f"Splicing_DM: {len(splice_mutations)}, Non-splicing_DM: {len(non_splice_mutations)}")

# Convert startCoord to hg19
from liftover import get_lifter

def _liftover_to_hg19(chrom, pos):
    converter = get_lifter('hg38', 'hg19')
    result = converter.query(chrom, pos)
    if result:
        return result[0]
    else:
        return None
    
def anno_hg19_pos(row):
    converted = _liftover_to_hg19(row['CHROM'], row['POS_hg38'])
    if converted:
        return converted[0]
    else:
        return None

allmut_dm_maf0_snv['POS_hg19'] = allmut_dm_maf0_snv.parallel_apply(anno_hg19_pos, axis=1)
# allmut_dm_maf0_snv.to_pickle('allmut_dm_maf0_snv_liftover.pkl', mode='x')

### Liftover処理後（ここから解析すればOK）

In [2]:
# Loading allmut variants from pickle
allmut_dm_maf0_snv_hg19 = pd.read_pickle('allmut_dm_maf0_snv_liftover.pkl')

# Rename POS_hg19 to POS
allmut_dm_maf0_snv_hg19.rename(columns={'POS_hg19': 'POS'}, inplace=True)

# Drop unknown positions in 'POS' column and assign integer type
allmut_dm_maf0_snv_hg19.dropna(subset=['POS'], inplace=True)
allmut_dm_maf0_snv_hg19 = allmut_dm_maf0_snv_hg19.astype({'POS': int})

# Change object name to allmut
allmut = allmut_dm_maf0_snv_hg19

# Generate ID column
allmut['ID'] = allmut['CHROM'].astype(str) + '-' + allmut['POS'].astype(str) + '-' + allmut['hgvs']

# Extract useful columns
allmut = allmut[['ID', 'mutype', 'clinvar_clnsig', 'tag', 'deletion', 'insertion', 'expected_inheritance', 'gnomad_AF']]

# Load VCF file annoteted by analysis pipeline
df = pd.read_pickle('splai_vep_vcfs/hgmd_dm/allchr.DM.splai.vep.nondel.enst.prescore.hgnconly.v2.pkl')
df['HGVSc'] = df['HGVSc'].str.replace('c.', '')
df['ID'] = df['CHROM'].astype(str) + '-' + df['POS'].astype(str) + '-' + df['HGVSc']

# merge df and allmut on 'ID' column with inner join
print(len(df))
df = pd.merge(df, allmut, on='ID', how='inner')
print(len(df))

exclude_csq = {
    '3_prime_UTR_variant', '5_prime_UTR_variant', 'mature_miRNA_variant',
    'mature_miRNA_variant', 'downstream_gene_variant', 'upstream_gene_variant'
}

def is_orf_variants(row):
    csqs: list = row['Consequence'].split('&')
    if set(csqs).isdisjoint(exclude_csq):
        return True
    else:
        return False

df['is_ORF'] = df.apply(is_orf_variants, axis=1)
df = df[df['is_ORF']]

# df_spl contains splicing mutations (splice, canonical-splice, exonic-splice)
df_spl = df[df['mutype'].str.contains('splice')].copy()

# df_non_spl contains non-splicing mutations (missense, nonsense, synonymous)
df_non_spl = df[df['mutype'].str.contains('missense|nonsense|synonymous')].copy()

print(f"Splicing: {len(df_spl)}, Non-splicing: {len(df_non_spl)}, total: {len(df_spl) + len(df_non_spl)}")

256603
126859
Splicing: 18432, Non-splicing: 107021, total: 125453


In [3]:
# Save as tsv
df_spl.to_csv('excel/allchr.DM.splai.vep.nondel.enst.prescore.hgnconly.spl.tsv', sep='\t', index=False)
df_non_spl.to_csv('excel/allchr.DM.splai.vep.nondel.enst.prescore.hgnconly.nonspl.tsv', sep='\t', index=False)

In [4]:
# Annotating the label and variant_id (CHROM-POS-REF_ALT)
# When mutype is splice, the label is 1, otherwise 0
df_gnomad = pd.read_pickle('splai_vep_vcfs/gnomadv211/allchr.gnomad.splai.vep.vcf.enst.prescore.hgnconly.v2.pkl')
df_gnomad['is_ORF'] = df_gnomad.apply(is_orf_variants, axis=1)
df_gnomad = df_gnomad[df_gnomad['is_ORF']]

df_spl['LABEL'] = 1
df_non_spl['LABEL'] = 0
df_gnomad['LABEL'] = 0

df_spl['variant_id'] = df_spl['CHROM'].astype(str) + '-' + df_spl['POS'].astype(str) + '-' + df_spl['REF'] + '-' + df_spl['ALT']
df_non_spl['variant_id'] = df_non_spl['CHROM'].astype(str) + '-' + df_non_spl['POS'].astype(str) + '-' + df_non_spl['REF'] + '-' + df_non_spl['ALT']
df_gnomad['variant_id'] = df_gnomad['CHROM'].astype(str) + '-' + df_gnomad['POS'].astype(str) + '-' + df_gnomad['REF'] + '-' + df_gnomad['ALT']

# Create a dataframe tp
tp = df_spl.copy()

# Concatenate df_gnomad and df_non_spl
tn = pd.concat([df_gnomad, df_non_spl], ignore_index=True)
tn = tn.drop_duplicates(subset=['variant_id'], keep='first')

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import os, sys
from pathlib import Path
from pandarallel import pandarallel
from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

########   Initialize and setup pandas methods   ########
pandarallel.initialize(nb_workers=7, progress_bar=False, 
                       verbose=2, use_memory_fs=False) 
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp' 

try: 
    __file__
    sys.path.append(os.path.join(os.path.dirname('__file__')))
except NameError:
    Path().resolve()
    sys.path.append(os.path.join(Path().resolve(), '../../'))

from libs.scoring import Scoring

import warnings
warnings.simplefilter('ignore')

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
def specificity_sensitivity_plotly(data):
    thresholds = np.arange(0, 11, 1)
    results = []

    for threshold in thresholds:
        tp = data[(data['PriorityScore'] >= threshold) & (data['LABEL'] == 1)].shape[0]
        fn = data[(data['PriorityScore'] < threshold) & (data['LABEL'] == 1)].shape[0]
        tn = data[(data['PriorityScore'] < threshold) & (data['LABEL'] == 0)].shape[0]
        fp = data[(data['PriorityScore'] >= threshold) & (data['LABEL'] == 0)].shape[0]
        specificity = tn / (tn + fp) if (tn + fp) else 0
        sensitivity = tp / (tp + fn) if (tp + fn) else 0
        # print(f"Threshold: {threshold}, TP: {tp}, FN: {fn}, TN: {tn}, FP: {fp}")
        # print(f"Threshold: {threshold}, Specificity: {specificity:.6f}, Sensitivity: {sensitivity:.6f}")
        results.append({'Threshold': threshold, 'Metric': 'Specificity', 'Value': specificity})
        results.append({'Threshold': threshold, 'Metric': 'Sensitivity', 'Value': sensitivity})

    results_df = pd.DataFrame(results)
    return results_df

def plot_sensitivity_specificity_plotly(
        results_df: pd.DataFrame, w: int, h: int):
    # Separate the dataframes for specificity and sensitivity
    specificity_df = results_df[results_df['Metric'] == 'Specificity']
    sensitivity_df = results_df[results_df['Metric'] == 'Sensitivity']

    # Plotly Graph Objectsを使用してプロット
    fig = go.Figure()

    # 特異性
    fig.add_trace(go.Scatter(
        x=specificity_df['Threshold'],
        y=specificity_df['Value'],
        marker=dict(color='#665990'),
        mode='lines+markers',
        name='Specificity',
        text=[f'Threshold: {th}, Specificity: {val:.3f}' for th, val in zip(specificity_df['Threshold'], specificity_df['Value'])],
        hoverinfo='text'
    ))
    
    # 感度
    fig.add_trace(go.Scatter(
        x=sensitivity_df['Threshold'],
        y=sensitivity_df['Value'],
        marker=dict(color='#F8ACAC'),
        mode='lines+markers',
        name='Sensitivity',
        text=[f'Threshold: {th}, Sensitivity: {val:.3f}' for th, val in zip(sensitivity_df['Threshold'], sensitivity_df['Value'])],
        hoverinfo='text'
    ))

    # Y軸のフォーマット設定
    fig.update_yaxes(tickformat=".1f")

    # グラフのレイアウト設定
    fig.update_layout(title='Sensitivity and Specificity for each threshold',
                      xaxis_title='Threshold',
                      yaxis_title='Sensitivity/Specificity',
                      plot_bgcolor='rgba(243, 243, 243, 1)',
                      paper_bgcolor='rgba(243, 243, 243, 0)',
                      font=dict(family="Arial, sans-serif", size=12, color="black"),
                      legend=dict(y=0.075, x=0.75, xanchor='right', yanchor='bottom', 
                              bgcolor='rgba(243, 243, 243, 1)',
                              font=dict(family="Arial, sans-serif", size=12, color="black"))
                              )

    # グラフサイズの調整
    fig.update_layout(width=w, height=h)
    fig.write_html("sensitivity_specificity_plot.html")

    # fig.show()
    return fig

def plot_sensitivity_specificity_plotly_without_legened(results_df):
    # Separate the dataframes for specificity and sensitivity
    specificity_df = results_df[results_df['Metric'] == 'Specificity']
    sensitivity_df = results_df[results_df['Metric'] == 'Sensitivity']

    # Plotly Graph Objectsを使用してプロット
    fig = go.Figure()

    # 特異性
    fig.add_trace(go.Scatter(
        x=specificity_df['Threshold'],
        y=specificity_df['Value'],
        marker=dict(color='green'),
        mode='lines+markers',
        name='Specificity',
        text=[f'Threshold: {th}, Specificity: {val:.8f}' for th, val in zip(specificity_df['Threshold'], specificity_df['Value'])],
        hoverinfo='text',
        showlegend=False 
    ))
    
    # 感度
    fig.add_trace(go.Scatter(
        x=sensitivity_df['Threshold'],
        y=sensitivity_df['Value'],
        marker=dict(color='orange'),
        mode='lines+markers',
        name='Sensitivity',
        text=[f'Threshold: {th}, Sensitivity: {val:.8f}' for th, val in zip(sensitivity_df['Threshold'], sensitivity_df['Value'])],
        hoverinfo='text',
        showlegend=False
    ))

    # Y軸のフォーマット設定
    fig.update_yaxes(tickformat=".1f")

    # グラフのレイアウト設定
    fig.update_layout(title='Sensitivity and Specificity for each threshold',
                      xaxis_title='Threshold',
                      yaxis_title='Sensitivity/Specificity',
                      plot_bgcolor='rgba(243, 243, 243, 1)',
                      paper_bgcolor='rgba(243, 243, 243, 0)',
                      font=dict(family="Arial, sans-serif", size=12, color="black"),
                      legend=dict(y=0.075, x=0.75, xanchor='right', yanchor='bottom', 
                              bgcolor='rgba(243, 243, 243, 1)',
                              font=dict(family="Arial, sans-serif", size=12, color="black"))
                              )

    # グラフサイズの調整
    fig.update_layout(width=600, height=600)
    fig.write_html("sensitivity_specificity_plot.html")

    # fig.show()
    return fig


# Code below is adapted from Netflix's VMAF and BesenbacherLab's ROC-utils
# https://github.com/Netflix/vmaf/
# https://github.com/BesenbacherLab/ROC-utils
# Modifications: np.float -> np.float64

def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float64)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float64)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float64)
    ty = np.empty([k, n], dtype=np.float64)
    tz = np.empty([k, m + n], dtype=np.float64)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    return order, label_1_count


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

# Calculate AUC confidence interval (95%)
def compute_auc_confidence_interval(auc, var, confidence_level=0.95):
    alpha = 1 - confidence_level
    z_score = scipy.stats.norm.ppf(1 - alpha/2)  # 2-tailed z score
    se = np.sqrt(var)  # Calculate SE from variance
    lower_bound = auc - z_score * se
    upper_bound = auc + z_score * se
    return lower_bound, upper_bound


from ortools.sat.python import cp_model

class SolutionCollector(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solutions = []

    def OnSolutionCallback(self):
        solution = {v.Name(): self.Value(v) for v in self.__variables}
        self.__solutions.append(solution)

    def GetAllSolutions(self):
        return self.__solutions

def find_all_solutions():
    # モデルを初期化
    model = cp_model.CpModel()

    # 変数の定義
    s = {i: model.NewIntVar(-10, 10, f's{i}') for i in range(1, 15)}

    # 制約の追加
    model.Add(s[3] == 0)                    #
    model.Add(s[1] + s[10] + s[14] == 9)    #
    model.Add(s[3] + s[11] + s[12] == 0)    #
    # model.Add(s[1] + s[7] < 9)
    # model.Add(s[10] >= 3)
    # model.Add(s[10] <= 7)
    
    # Knowledge-based
    # Absolutes
    model.Add(s[2] >= 1)                #
    model.Add(s[1] - s[2] > 0)          #   
    
    # SpliceAI limitation
    # Absolutes
    model.Add(s[4] < 0)                 #
    model.Add(s[5] <= 0)                #
    model.Add(s[6] >= 0)                #
    # model.Add(s[7] >= 1)
    model.Add(s[12] < 0)                #
    model.Add(s[13] <= 0)               #
    model.Add(s[14] >= 0)               #
    # Order
    model.Add(s[5] - s[4] > 0)          #
    model.Add(s[6] - s[5] > 0)          #
    model.Add(s[7] - s[6] > 0)          #
    model.Add(s[13] - s[12] > 0)        #
    model.Add(s[14] - s[13] > 0)        #
    model.Add(s[4] >= s[12])            #

    # under limit
    # model.Add(s[1] + s[10] + s[12] >= 0)
    # model.Add(s[4] <= s[10] + s[12])

    # Absolutes
    model.Add(s[11] >= 0)               #
    # Score order
    model.Add(s[9] >= s[7])             #
    model.Add(s[8] - s[9] > 0)          #
    model.Add(s[10] - s[8] > 0)         #
    # model.Add(s[9] - s[11] > 0)         #
    model.Add(s[9] - s[11] >= 0)         #


    # ソルバの設定と解の探索
    solver = cp_model.CpSolver()
    solution_collector = SolutionCollector([s[i] for i in range(1, 15)])
    solver.SearchForAllSolutions(model, solution_collector)
    
    # 全ての解を返す
    return solution_collector.GetAllSolutions()

# 解の計算
all_solutions = find_all_solutions()
print(f'Total solutions found: {len(all_solutions)}')

# Debug and confirm the solutions
# for index, solution in enumerate(all_solutions):
#     print(f'Solution {index + 1}: {solution}')



Total solutions found: 4850


In [7]:
frac = 0.15
random_state = 99

In [8]:
# prepare the results
# tp['is_Canonical'].replace({True: "Yes", False: "No"}, inplace=True)
# tn['is_Canonical'].replace({True: "Yes", False: "No"}, inplace=True)

# Split the data into training and test sets
tp_train = tp.sample(frac=frac, random_state=random_state)
tp_test = tp.drop(tp_train.index)
tn_train = tn.sample(frac=frac, random_state=random_state)
tn_test = tn.drop(tn_train.index)

# Save the dataframes as pickle files
tp_train.to_pickle(f'train_test_pkls/tp_prescore_train_{random_state}.pkl')
tp_test.to_pickle(f'train_test_pkls/tp_prescore_test_{random_state}.pkl')
tn_train.to_pickle(f'train_test_pkls/tn_prescore_train_{random_state}.pkl')
tn_test.to_pickle(f'train_test_pkls/tn_prescore_test_{random_state}.pkl')

### Find optimal weights

In [ ]:
results = []
buf: float = 0.978

for i, solution in enumerate(all_solutions):
    ths_scores = {'clinvar_same_pos': solution['s1'],
            'clinvar_same_motif': solution['s2'],
            'clinvar_else': solution['s3'],
            'non_canon_splai_lte_0.1_outside': solution['s4'],    
            'non_canon_splai_lte_0.1_other': solution['s5'],
            'non_canon_splai_bet_0.1_0.2': solution['s6'],
            'non_canon_splai_gte_0.2': solution['s7'],
            'canon_strong': solution['s8'], 
            'canon_moderate': solution['s9'], 
            'frameshift_nmd_eloF': solution['s10'], 
            'frameshift_nmd_not_eloF': solution['s11'],
            'canon_splai_lte_0.1': solution['s12'],
            'canon_splai_bet_0.1_0.2': solution['s13'],
            'canon_splai_gte_0.2': solution['s14']
            }
    
    scoring = Scoring(ths=ths_scores)

    tp_train = pd.read_pickle(f'train_test_pkls/tp_prescore_train_{random_state}.pkl')
    tn_train = pd.read_pickle(f'train_test_pkls/tn_prescore_train_{random_state}.pkl')

    tp_train['insilico_screening'] = tp_train.parallel_apply(scoring.insilico_screening, axis=1)
    tp_train['clinvar_screening'] = tp_train.parallel_apply(scoring.clinvar_screening, axis=1)
    tp_train = tp_train[tp_train['insilico_screening'] != 'Not available']
    tp_train['PriorityScore'] = tp_train['insilico_screening'] + tp_train['clinvar_screening']

    # tp_train['PriorityScore'] = tp_train.parallel_apply(scoring.calc_priority_score, axis=1)
    # tp_train = scoring.calc_priority_score(tp_train)

    tn_train['insilico_screening'] = tn_train.parallel_apply(scoring.insilico_screening, axis=1)
    tn_train['clinvar_screening'] = tn_train.parallel_apply(scoring.clinvar_screening, axis=1)
    tn_train = tn_train[tn_train['insilico_screening'] != 'Not available']
    tn_train['PriorityScore'] = tn_train['insilico_screening'] + tn_train['clinvar_screening']
    # tn_train['PriorityScore'] = tn_train.parallel_apply(scoring.calc_priority_score, axis=1)
    # tn_train = scoring.calc_priority_score(tn_train)

    # Extract the columns needed
    tp_train = tp_train[['variant_id', 'LABEL', 'PriorityScore', 'maxsplai']]
    tn_train = tn_train[['variant_id', 'LABEL', 'PriorityScore', 'maxsplai']]

    ### ========================================================== ##
    data = pd.concat([tp_train, tn_train], ignore_index=True)
    data.drop_duplicates(subset='variant_id', keep=False, inplace=True)

    # Cast the columns to float type
    data['LABEL'] = data['LABEL'].astype(int)
    # Extract rows with PriorityScore not 'Not available'
    data = data[data['PriorityScore'] != 'Not available']
    data['PriorityScore'] = data['PriorityScore'].astype(float)
    data['maxsplai'] = data['maxsplai'].astype(float)

    ## DeLong test and AUC confidence interval
    ground_truth = np.array(data['LABEL'])
    predictions_fw = np.array(data['PriorityScore'])

    auc1, var1 = delong_roc_variance(ground_truth, predictions_fw)
    cilower1, ciupper1 = compute_auc_confidence_interval(auc1, var1)

    results.append(
        {'index': i+1, 's1': solution['s1'], 's2': solution['s2'], 
         's3': solution['s3'], 's4': solution['s4'], 's5': solution['s5'], 
         's6': solution['s6'], 's7': solution['s7'], 's8': solution['s8'], 
         's9': solution['s9'], 's10': solution['s10'], 's11': solution['s11'], 
         's12': solution['s12'], 's13': solution['s13'], 's14': solution['s14'],
         'auROC': f"{auc1:.10f}, '95% Confidence Interval': {cilower1:.12f}-{ciupper1:.12f}"
        }
    )

    if auc1 > buf:
        buf = auc1
        print(f"\n===== New best AUC: {auc1:.10f} with solution {i+1} =======")
        print(f"New best solution {i}: {solution} \n")
        predictions_sp = np.array(data['maxsplai'])
        auc2, var2 = delong_roc_variance(ground_truth, predictions_sp)
        cilower2, ciupper2 = compute_auc_confidence_interval(auc2, var2)
        p_value_log = delong_roc_test(ground_truth, predictions_fw, predictions_sp)
        print(f"AUC - Framework (95%CI): {auc1:.3f} [{cilower1:.4f}-{ciupper1:.4f}]")
        print(f"AUC - SpliceAI (95%CI) : {auc2:.3f} [{cilower2:.4f}-{ciupper2:.4f}]")
        print(f"p-value (DeLong Test)  : {10**p_value_log[0][0]:.2e}\n")
        print("===========================================================")
        
    if i % 10 == 0:
        print(f"###  Processed {i} solutions  ###")


###  Processed 0 solutions  ###
###  Processed 10 solutions  ###
###  Processed 20 solutions  ###
###  Processed 30 solutions  ###
###  Processed 40 solutions  ###
###  Processed 50 solutions  ###
###  Processed 60 solutions  ###
###  Processed 70 solutions  ###
###  Processed 80 solutions  ###
###  Processed 90 solutions  ###
###  Processed 100 solutions  ###
###  Processed 110 solutions  ###
###  Processed 120 solutions  ###
###  Processed 130 solutions  ###
###  Processed 140 solutions  ###
###  Processed 150 solutions  ###
###  Processed 160 solutions  ###
###  Processed 170 solutions  ###
###  Processed 180 solutions  ###
###  Processed 190 solutions  ###
###  Processed 200 solutions  ###
###  Processed 210 solutions  ###
###  Processed 220 solutions  ###
###  Processed 230 solutions  ###
###  Processed 240 solutions  ###
###  Processed 250 solutions  ###
###  Processed 260 solutions  ###
###  Processed 270 solutions  ###
###  Processed 280 solutions  ###
###  Processed 290 soluti

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x108363700>>
Traceback (most recent call last):
  File "/Volumes/SSD_480GB/utsu/miniconda3/envs/nar/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


###  Processed 2470 solutions  ###


#### Performance test using test variant set

In [16]:
thsdict = {
    'opti': 
            {'clinvar_same_pos': 2,
             'clinvar_same_motif': 1,
             'clinvar_else': 0,
             'non_canon_splai_lte_0.1_outside': -3,
             'non_canon_splai_lte_0.1_other': -2,
             'non_canon_splai_bet_0.1_0.2': 1,
             'non_canon_splai_gte_0.2': 2,
             'canon_strong': 6, 
             'canon_moderate': 5, 
             'frameshift_nmd_eloF': 7, 
             'frameshift_nmd_not_eloF': 3,
             'canon_splai_lte_0.1': -3,
             'canon_splai_bet_0.1_0.2': -1,
             'canon_splai_gte_0.2': 0},
}

# thsdict = {
#     'opti': 
#             {'clinvar_same_pos': 2,
#              'clinvar_same_motif': 1,
#              'clinvar_else': 0,
#              'non_canon_splai_lte_0.1_outside': -1,
#              'non_canon_splai_lte_0.1_other': 0,
#              'non_canon_splai_bet_0.1_0.2': 1,
#              'non_canon_splai_gte_0.2': 3,
#              'canon_strong': 5, 
#              'canon_moderate': 4, 
#              'frameshift_nmd_eloF': 6, 
#              'frameshift_nmd_not_eloF': 3,
#              'canon_splai_lte_0.1': -3,
#              'canon_splai_bet_0.1_0.2': 0,
#              'canon_splai_gte_0.2': 1},
# }


# Memo
# {'s1': 2, 's2': 1, 's3': 0, 's4': -1, 's5': 0, 's6': 1, 's7': 3, 's8': 5, 's9': 4, 's10': 6, 's11': 2, 's12': -2, 's13': 0, 's14': 1}
# {'s1': 2, 's2': 1, 's3': 0, 's4': -1, 's5': 0, 's6': 1, 's7': 3, 's8': 5, 's9': 4, 's10': 6, 's11': 2, 's12': -2, 's13': 0, 's14': 1}
# {'s1': 2, 's2': 1, 's3': 0, 's4': -1, 's5': 0, 's6': 1, 's7': 3, 's8': 5, 's9': 4, 's10': 6, 's11': 3, 's12': -3, 's13': 0, 's14': 1}

# thsdict = {
#     'opti': 
#             {'clinvar_same_pos': 3,
#              'clinvar_same_motif': 2,
#              'clinvar_else': 0,
#              'non_canon_splai_lte_0.1_outside': -4,
#              'non_canon_splai_lte_0.1_other': -3,
#              'non_canon_splai_bet_0.1_0.2': 3,
#              'non_canon_splai_gte_0.2': 4,
#              'canon_strong': 5, 
#              'canon_moderate': 4, 
#              'frameshift_nmd_eloF': 6, 
#              'frameshift_nmd_not_eloF': 4,
#              'canon_splai_lte_0.1': -4,
#              'canon_splai_bet_0.1_0.2': -1,
#              'canon_splai_gte_0.2': 0},
# }

tp_test = pd.read_pickle(f'train_test_pkls/tp_prescore_test_{random_state}.pkl')
tn_test = pd.read_pickle(f'train_test_pkls/tn_prescore_test_{random_state}.pkl')


# Exclude not available values
tp_test = tp_test[tp_test['ENST_Full'] != "[Warning] ENST_with_Ver_not_available"]
tn_test = tn_test[tn_test['ENST_Full'] != "[Warning] ENST_with_Ver_not_available"]
tp_test = tp_test[tp_test['maxsplai'] != "NA"]
tn_test = tn_test[tn_test['maxsplai'] != "NA"]

In [13]:
ths = thsdict['opti']
scoring = Scoring(ths=ths)

tp_test['insilico_screening'] = tp_test.parallel_apply(scoring.insilico_screening, axis=1)
tp_test['clinvar_screening'] = tp_test.parallel_apply(scoring.clinvar_screening, axis=1)
tp_test['PriorityScore'] = tp_test.parallel_apply(scoring.calc_priority_score, axis=1)
# tp_test = scoring.calc_priority_score2(tp_test)


tn_test['insilico_screening'] = tn_test.parallel_apply(scoring.insilico_screening, axis=1)
tn_test['clinvar_screening'] = tn_test.parallel_apply(scoring.clinvar_screening, axis=1)
tn_test['PriorityScore'] = tn_test.parallel_apply(scoring.calc_priority_score, axis=1)
# tn_test = scoring.calc_priority_score2(tn_test)

######### save as tsv 
tp_test.to_csv('excel/tp_test_score.tsv', sep='\t', index=False)
tn_test.to_csv('excel/tn_test_score.tsv', sep='\t', index=False)
#########

# Extract the columns needed
tp_test = tp_test[['variant_id', 'LABEL', 'PriorityScore', 'maxsplai']]
tn_test = tn_test[['variant_id', 'LABEL', 'PriorityScore', 'maxsplai']]

### ========================================================== ##
data = pd.concat([tp_test, tn_test], ignore_index=True)
data.drop_duplicates(subset='variant_id', keep=False, inplace=True)

# Cast the columns to float type
data['LABEL'] = data['LABEL'].astype(int)
data = data[data['PriorityScore'] != 'Not available']   # Exclude rows with PriorityScore not 'Not available'
data['PriorityScore'] = data['PriorityScore'].astype(float)
data['maxsplai'] = data['maxsplai'].astype(float)

# Plot the sensitivity and specificity for each threshold
results_df = specificity_sensitivity_plotly(data)
fig_opti = plot_sensitivity_specificity_plotly(results_df, 800, 800)
# fig2 = plot_sensitivity_specificity_plotly_without_legened(results_df)
# print(tp_test['PriorityScore'].isnull().sum(), tn_test['PriorityScore'].isnull().sum())

## DeLong test and AUC confidence interval
ground_truth = np.array(data['LABEL'])
predictions_fw = np.array(data['PriorityScore'])
predictions_sp = np.array(data['maxsplai'])

auc1, var1 = delong_roc_variance(ground_truth, predictions_fw)
cilower1, ciupper1 = compute_auc_confidence_interval(auc1, var1)
auc2, var2 = delong_roc_variance(ground_truth, predictions_sp)
cilower2, ciupper2 = compute_auc_confidence_interval(auc2, var2)

p_value_log = delong_roc_test(ground_truth, predictions_fw, predictions_sp)

print(f"AUC - Framework (95%CI): {auc1:.3f} [{cilower1:.4f}-{ciupper1:.4f}]")
print(f"AUC - SpliceAI (95%CI) : {auc2:.3f} [{cilower2:.4f}-{ciupper2:.4f}]")
print(f"p-value (DeLong Test)  : {10**p_value_log[0][0]:.2e}")


AUC - Framework (95%CI): 0.971 [0.9677-0.9745]
AUC - SpliceAI (95%CI) : 0.978 [0.9749-0.9813]
p-value (DeLong Test)  : 1.18e-07


In [14]:
def specificity_sensitivity_plotly(data):
    thresholds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    results = []

    for threshold in thresholds:
        tp = data[(data['PriorityScore'] >= threshold) & (data['LABEL'] == 1)].shape[0]
        fn = data[(data['PriorityScore'] < threshold) & (data['LABEL'] == 1)].shape[0]
        tn = data[(data['PriorityScore'] < threshold) & (data['LABEL'] == 0)].shape[0]
        fp = data[(data['PriorityScore'] >= threshold) & (data['LABEL'] == 0)].shape[0]
        specificity = tn / (tn + fp) if (tn + fp) else 0
        sensitivity = tp / (tp + fn) if (tp + fn) else 0
        results.append({'Threshold': threshold, 'Metric': 'Specificity', 'Value': specificity})
        results.append({'Threshold': threshold, 'Metric': 'Sensitivity', 'Value': sensitivity})

    return results_df

import plotly.graph_objects as go
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score

# ROC curve (LABEL, PriorityScore)
fpr1, tpr1, thresholds1 = roc_curve(data['LABEL'], data['PriorityScore'])
auc1 = roc_auc_score(data['LABEL'], data['PriorityScore'])

fpr2, tpr2, thresholds2 = roc_curve(data['LABEL'], data['maxsplai'])
auc2 = roc_auc_score(data['LABEL'], data['maxsplai'])

# Calculate optimal threshold from ROC curve by Youden's J statistic
Youden_index = np.argmax(tpr1 - fpr1)
optimal_threshold = thresholds1[Youden_index]
print('Optimal threshold (using Youden index):', optimal_threshold)

# plot ROC curve using Plotly
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(
    x=fpr1, y=tpr1, mode='lines', 
    name=f"Framework      ({auc1:.3f} [{cilower1:.4f}-{ciupper1:.4f}])", 
    line=dict(color='#E41A1C', width=2))
    )
fig.add_trace(go.Scatter(
    x=fpr2, y=tpr2, mode='lines', 
    name=f"SpliceAI Alone ({auc2:.3f} [{cilower2:.4f}-{ciupper2:.4f}])", 
    line=dict(color='#377EB8', width=2))
    )
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines', name='Chance', 
    line=dict(color='gray', width=2, dash='dash'), showlegend=False)
    )

# Add an annotation
fig.add_annotation(x=0.6, y=0.05, xref="paper", yref="paper",
                   text=f"DeLong's test p-value = {10**p_value_log[0][0]:.2e}",
                   showarrow=False,
                   font=dict(family="Arial, sans-serif", size=12, color="black"),
                   bgcolor='rgba(243, 243, 243, 1)',
                #    bordercolor="black",
                   borderwidth=2)

# Add titles and labels
fig.update_layout(title='ROC Curve Comparison',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate',
                  legend_title='Prediction methods (AUC [95%CI])',
                  plot_bgcolor='rgba(243, 243, 243, 1)',
                  paper_bgcolor='rgba(243, 243, 243, 0)',
                  legend=dict(y=0.09, x=0.925, xanchor='right', yanchor='bottom', 
                              bgcolor='rgba(243, 243, 243, 1)',
                              font=dict(family="Arial, sans-serif", size=12, color="black")),
                  margin=dict(l=40, r=40, t=40, b=40))

fig.update_xaxes(range=[-0.05, 1.05])
fig.update_yaxes(range=[-0.05, 1.05])
fig.update_layout(width=600, height=600)
fig.write_html("roc-auc.html")

# Add zoomed-in ROC curve in the top left corner
fig.add_trace(go.Scatter(
    x=fpr1, y=tpr1, mode='lines', 
    name=f"Framework      ({auc1:.3f} [{cilower1:.4f}-{ciupper1:.4f}])", 
    line=dict(color='#E41A1C', width=2), showlegend=False)
    )


# Show figure
fig.show()


Optimal threshold (using Youden index): 3.0


In [15]:
# Compare the performance of the two models
# Confusion matrix
th_4, th_5 = 4, 5
th_02, th_05, th_08 = 0.2, 0.5, 0.8
thresholds_fw = [th_4, th_5]
thresholds_spl = [th_02, th_05, th_08]

def calculate_performance_metrics_fw(data, threshold):
    data['Prediction'] = data['PriorityScore'] >= threshold
    tn, fp, fn, tp = confusion_matrix(data['LABEL'], data['Prediction']).ravel()
    specificity = tn / (tn + fp) if (tn + fp) else 0
    sensitivity = tp / (tp + fn) if (tp + fn) else 0
    accuracy = accuracy_score(data['LABEL'], data['Prediction'])
    precision = precision_score(data['LABEL'], data['Prediction'])
    f1 = f1_score(data['LABEL'], data['Prediction'])
    return specificity, sensitivity, accuracy, precision, f1

def calculate_performance_metrics_spl(data, threshold):
    data['Prediction'] = data['maxsplai'] >= threshold
    tn, fp, fn, tp = confusion_matrix(data['LABEL'], data['Prediction']).ravel()
    specificity = tn / (tn + fp) if (tn + fp) else 0
    sensitivity = tp / (tp + fn) if (tp + fn) else 0
    accuracy = accuracy_score(data['LABEL'], data['Prediction'])
    precision = precision_score(data['LABEL'], data['Prediction'])
    
    f1 = f1_score(data['LABEL'], data['Prediction'])
    return specificity, sensitivity, accuracy, precision, f1

performance_metrics = []
for threshold in thresholds_fw:
    specificity, sensitivity, accuracy, precision, f1 = calculate_performance_metrics_fw(data, threshold)
    performance_metrics.append({'Threshold': threshold, 'Specificity': specificity, 'Sensitivity': sensitivity, 'Accuracy': accuracy, 'Precision': precision, 'F1': f1})

for threshold in thresholds_spl:
    specificity, sensitivity, accuracy, precision, f1 = calculate_performance_metrics_spl(data, threshold)
    performance_metrics.append({'Threshold': threshold, 'Specificity': specificity, 'Sensitivity': sensitivity, 'Accuracy': accuracy, 'Precision': precision, 'F1': f1})

performance_metrics_df = pd.DataFrame(performance_metrics)
columns = ['Category', 'Specificity', 'Sensitivity', 'Accuracy', 'Precision', 'F1']
performance_metrics_df.columns = columns
performance_metrics_df.replace(
    {'Category': {
        4.0: 'Framework (4)<br> High sensitivity', 
        5.0: 'Framework (5)<br> High specificity', 
        0.2: 'SpliceAI (0.2)<br> High sensitivity', 
        0.5: 'SpliceAI (0.5)<br> Recommended', 
        0.8: 'SpliceAI (0.8)<br> High precision'
        }}, inplace=True)

# print(performance_metrics_df)

colors = ['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3']

fig = go.Figure()

# Add traces with ggplot2 style
fig.add_trace(go.Bar(x=performance_metrics_df['Category'], y=performance_metrics_df['Specificity'], name='Specificity', marker_color=colors[0]))
fig.add_trace(go.Bar(x=performance_metrics_df['Category'], y=performance_metrics_df['Sensitivity'], name='Sensitivity', marker_color=colors[2]))
# fig.add_trace(go.Bar(x=performance_metrics_df['Category'], y=performance_metrics_df['Accuracy'], name='Accuracy', marker_color=colors[2]))
fig.add_trace(go.Bar(x=performance_metrics_df['Category'], y=performance_metrics_df['Precision'], name='Precision', marker_color=colors[3]))
fig.add_trace(go.Bar(x=performance_metrics_df['Category'], y=performance_metrics_df['F1'], name='F-score', marker_color=colors[4]))

# Add titles and labels with ggplot2 style using bar plot
fig.update_layout(title='Performance Metrics Comparison',
                  xaxis_title='Methods (cutoff)',
                  yaxis_title='Value',
                  legend_title='Performance Metrics',
                  plot_bgcolor='rgba(243, 243, 243, 1)',
                  paper_bgcolor='rgba(243, 243, 243, 0)',
                  legend=dict(y=0.075, x=0.95, xanchor='right', yanchor='bottom', 
                              bgcolor='rgba(255, 255, 255, 0.8)',
                              bordercolor='rgba(0, 0, 0, 0)',
                              font=dict(family="Arial, sans-serif", size=12, color="black")),
                  margin=dict(l=40, r=40, t=40, b=40)
                  )

# add annotations on top of the bars
font = dict(family="Arial, sans-serif", size=9, color="black")
# for i, row in performance_metrics_df.iterrows():
#     fig.add_annotation(x=row['Category'], y=row['Specificity'], text=f"{row['Specificity']:.3f}", showarrow=False, font=font, xshift=-42, yshift=10)
#     fig.add_annotation(x=row['Category'], y=row['Sensitivity'], text=f"{row['Sensitivity']:.3f}", showarrow=False, font=font, xshift=-14, yshift=10)
#     fig.add_annotation(x=row['Category'], y=row['Accuracy'], text=f"{row['Accuracy']:.3f}", showarrow=False, font=font, xshift=14, yshift=10)
# #     fig.add_annotation(x=row['Category'], y=row['Precision'], text=f"{row['Precision']:.3f}", showarrow=False, font=font, xshift=15, yshift=10)
#     fig.add_annotation(x=row['Category'], y=row['F1'], text=f"{row['F1']:.3f}", showarrow=False, font=font, xshift=42, yshift=10)

# fig.add_annotation(x='Framework (2)<br> High sensitivity', 
#                    y='Specificity', text="0.970", 
#                    showarrow=False, font=font, xshift=10, yshift=0)


fig.update_layout(barmode='group')
fig.update_xaxes(categoryorder='total ascending')
fig.update_layout(width=1000, height=600)

fig.update_xaxes(categoryorder='array')
                #  categoryarray=['FW_2', 'FW_3', 'SPL_0.2', 'SPL_0.5', 'SPL_0.8'])
fig.write_html("performance-metrics.html")

# Show figure
fig.show()

